In [9]:
import sqlite3
import pandas as pd

In [11]:
query = """
SELECT * FROM sensor_table

"""

with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)
    
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
65,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
66,184053,Creekwood Estates,32.909042,-97.124160
67,196323,Duck Creek Air Quality,32.960114,-96.692200
68,196421,Kilmichael Lane - PAII,32.977444,-96.776110
